<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# [Scénarios d'Initiation à la Statistique](https://github.com/wikistat/Exploration)

# Modèle Linéaire Général  (MLG), exemples en <a href="https://cran.r-project.org/"><img src="https://cran.r-project.org/Rlogo.svg" style="max-width: 40px; display: inline" alt="R"/></a>

**Résumé** Ce scénario présente trois exemples de modèle linéaire général. Le premier est une [analyse de variance / covariance](http://wikistat.fr/pdf/st-m-modlin-anacova.pdf) (modèle gaussien) pour une enquête marketing. Les deux suivants des exemples de [régression logistique](http://wikistat.fr/pdf/st-m-modlin-logit.pdf) (modèle binomial). le deuxième exemple analyse une enquête sur le travail (absence / présence) des femmes, le troisième propose de modéliser puis prévoir une variable clinique binaire: occurrence d'une pathologie cardiaque, par un ensemble de variables qualitatives et quantitatives. C'est l'illustration de différentes stratégies pour optimiser une [régression logistique](http://wikistat.fr/pdf/st-m-modlin-reglog.pdf) (modèle binomial) avec sélection de variables avec  minimisation du critère AIC, estimation de l'erreur de prévision, courbe ROC.


**Répondre aux questions en s'aidant des résultats des exécutions**.

## 1 Modèle gaussien: AnOVa *vs.* AnCoVa
L'étude de ces données marketing est déjà réalisée dans un [scénario avec SAS](http://www.math.univ-toulouse.fr/~besse/Wikistat/pdf/st-scenar-reg-GLM-SAS.pdf). Elle n'est pas complètement détaillée ici, seule les modèles complets sont repris à titre de comparaison. 

### 1.1 Données d'une enquête marketing
Les données [milkR.dat](https://www.math.univ-toulouse.fr/~besse/Wikistat/data/milkR.dat), extraites de Jobson (1991), sont issues d'une étude marketing visant à étudier l'impact de campagnes publicitaires sur les ventes de différents aliments. Un échantillon ou "panel" de familles a été constitué en tenant compte du lieu d'habitation ainsi que de la constitution de la famille. Chaque semaine, chacune de ces familles ont rempli un questionnaire décrivant les achats réalisés. Nous nous limitons ici à l'étude de l'impact sur la *consommation de lait* de quatre campagnes diffusées sur des chaînes locales de télévision. Quatre villes, une par campagne publicitaire, ont été choisies dans cinq différentes régions géographiques. Les consommations en lait par chacune des six familles par ville alors été mesurées (en dollars) après deux mois de campagne. 

Les données initiales se présentent sous la forme d'un tableau à 6 variables~:
la région géographique, les 4 consommations pour chacune des  villes ou campagnes publicitaires diffusées, la taille de la famille. 

Elles sont lues et réorganisées pour satisfaire à la structure classique d'un plan factoriel équilibré (donc complet) croisant trois facteurs~: région, taille de la famille, type de campagne. L'objectif est d'étudier l'effet du type de publicité sur la consommation des ménages. 

In [ ]:
# Lecture des données
dat1=read.table("milkR.dat",header=T,sep=",")
summary(dat1)

 Mise en forme des données par construction de la matrice du plan d'expérience (*design matrix*). 

In [ ]:
# Mise en forme
design = expand.grid(
    Pub=factor(1:4, labels=c("Pub1","Pub2","Pub3","Pub4")),
    Taille=factor(1:6),
    Region=factor(1:5, labels=c("R1","R2","R3","R4","R5")))
# vectorisation des données de consommation
# La taille est un facteur
Conso=as.vector(t(dat1[,2:5]))
dat2=data.frame(design,Conso)
summary(dat2)

Même chose mais en considérant que la variable "Taille" (de la famille) est une variable quantitative.

In [ ]:
# La taille est quantitative
dat3=data.frame(dat2[,-2],"Taille"=as.numeric(dat2[,2]))
summary(dat3)

Visualisation des données sous cette forme. 

**Q** Est-il possible de conclure à des effets des facteurs sur la consommation?

In [ ]:
car=as.integer(dat3[,"Region"])
plot(dat3[,"Taille"],dat3[,"Conso"],col=dat3[,"Pub"],xlab="Taille",ylab="Consommation",pch=car)
legend("topleft",pch=1:5,legend=c("Reg1","Reg2","Reg3","Reg4","Reg5"))
legend("bottomright",legend=c("Pub1","Pub2","Pub3","Pub4"),text.col=1:4)

**Q** et de choisir une campagne de publicié "optimale" par région?
### 1.3 Modèle d'AnOVa
La variable "taille" est considérée qualitative comme un facteur à 6 nivaux. L'approche rigoureuse consiste à prendre en compte directement toutes les variables du plan d'expérience. 

Une version simplifiée sans interaction:

In [ ]:
mod1=glm(Conso~Taille+Pub+Region,data=dat2,family=gaussian)
print(mod1)
anova(mod1,test="F")

Une version complète avec les interactions d'ordre 2; celle d'ordre 3 ne peut pas être estimée car les observations ne sont pas répliquées.

In [ ]:
mod2=glm(Conso~(Pub+Taille+Region)**2,data=dat2,family=gaussian)
print(mod2)
anova(mod2,test="F")

**Q** Quel modèle retenir ? 

**Q** Pouvez vous conclure sur le choix de la campagne de publicité optimale par région?

### 1.4 Modèle d'AnCoVa complet
Même chose en considérant la variable "taille" comme quantitative. 

**Q** Quel avantage à prendre cette option?

In [ ]:
mod3=glm(Conso~(Pub+Taille+Region)**2,data=dat3,family=gaussian)
print(mod3)
anova(mod3,test="F")

**Q** Comment choisir la campagne de publicité optimale?

**Q** Comparer avec les résultats (p-valeurs) de l'anova.

## 2 Modèle binomial: régression logistique de données d'enquête
### 2.1 Les données 
Les données étudiées (Jobson 1992) sont issues d’une enquête réalisée auprès de 200 femmes mariées du Michigan. Les variables qualitatives sont les suivantes:
- `THISYR`: la variable à expliquer, (`Woui`) si la femme travaille l'année en cours (`Wnon`) sinon;
- `LASTYR`: (`Loui`) si la femme travaille l'année précédente (`Lnon`) sinon;
- `CHILD1`: code la présence (`Boui`) ou l'absence (`Bnon`) d’un enfant de moins de 2 ans;
- `CHILD2`: présence (`Eoui`) ou absence (`Enon`) d’un enfant entre 2 et 6 ans;
- `ASCEND`: ascendance noire (`Anoi`) ou blanche  (`Abla`).

Les  autres  variables,  âge (`AGE`),  nombre  d’années  d’études (`EDUC`), revenu du mari (`HUBINC`) sont quantitatives.

**Attention** les données ne sont pas issues d'un plan expérimental orthogonal. Elles sont disponibles dans le fichier [jobpanel.dat](https://www.math.univ-toulouse.fr/~besse/Wikistat/data/jobpanel.dat).

Lire le fichier puis recoder les facteurs. 

In [ ]:
# Lecture des données:
type=c("character","character","numeric","numeric","numeric","character","character","character")
panel=read.table("jobpanel.dat",colClasses=type,header=TRUE)
summary(panel)

In [ ]:
# Codage explicite des facteurs
panel[,"THISYR"]=factor(panel[,1],levels=c("0","1"),labels=c("Wnon","Woui"))
panel[,"LASTYR"]=factor(panel[,2],levels=c("0","1"),labels=c("Lnon","Loui"))
panel[,"HUBINC"]=panel[,3]
panel[,"AGE"]=panel[,4]
panel[,"EDUC"]=panel[,5]
panel[,"CHILD1"]=factor(panel[,6],levels=c("0","1"),labels=c("Bnon","Boui"))
panel[,"CHILD2"]=factor(panel[,7],levels=c("0","1"),labels=c("Enon","Eoui"))
panel[,"ASCEND"]=factor(panel[,8],levels=c("0","1"),labels=c("Abla","Anoi"))
panel=panel[,-c(1:8)]
summary(panel)

### 2.2 Exploration
#### Unidimensionnelle
Vérifier les distributions des variables quantitatives, justifier la transformation.

In [ ]:
hist(panel[,"HUBINC"])
panel[,"LHUBINC"]=log(1+panel[,"HUBINC"])
hist(panel[,"LHUBINC"])
hist(panel[,"AGE"])
hist(panel[,"EDUC"])

#### Bidimensionnelle
Observer les liaisons entre les variables.

In [ ]:
plot(panel[,"AGE"],panel[,"LHUBINC"])
mosaicplot(THISYR~ASCEND,data=panel)
boxplot(AGE~THISYR,data=panel)
boxplot(LHUBINC~THISYR,data=panel)

#### Multidimensionnelle
Transformer  les  variables  quantitatives  en  qualitatives  avant  de  calculer l’analyse multiple des correspondances à l’aide de la librairie FactoMineR.

In [ ]:
# Transformations
panel[,"AGEq"]=cut(panel$AGE,breaks=quantile(panel[,"AGE"],probs = seq(0, 1, 1/3)),
               labels=c("Ag1","Ag2","Ag3"),include.lowest = TRUE)
panel[,"HUBINCq"]=cut(panel$HUBINC,breaks=quantile(panel[,"HUBINC"],probs=seq(0,1,1/3)),
               labels=c("HI1","HI2","HI3"),include.lowest=TRUE)
panel[,"EDUCq"]=cut(panel$EDUC,breaks=c(0,11,13,20),labels=c("Edu0","Edu1","Edu2"),include.lowest=TRUE)

In [ ]:
summary(panel)

In [ ]:
# AFCM 
library(FactoMineR)
afcm=MCA(panel[,c(1,2,6,7,8,10,11,12)],quali.sup=c(1,2),graph=F)
plot(afcm,habillage="quali",invisible="ind")

Tenter une interprétation des axes.
### 2.3 Modélisation par régression logistique
La  prévision  de  la  variable `THISYR` n’est pas l’objectif, celle-ci est d'ailleurs très mauvaise; l’objectif est plutôt l'explication de cette variable par les autres afin de mettre en évidence l’influence des différents facteurs.

Par ailleurs la variable `LASTYR` très liée à celle à expliquer n'est pas utilisée dans le smodèles pour se focaliser sur les factuers socio-économiques.

Le modèle complet sans interaction est estimé en utilisant soit les variables quantitatives, soit celles quantitatives transformées en qualitatives par découpage en classes.

#### Avec les variables quantitatives

In [ ]:
panel.glm=glm(THISYR~LHUBINC+AGE+EDUC+CHILD1+CHILD2+ASCEND,family=binomial,data=panel)
summary(panel.glm)
anova(panel.glm,test="Chi")

**Q** Que dire de la qualité d'ajustement du modèle?

**Q** Il ne s'agit pas d'un plan d'expérience orthogonal. Quelle conséquence pour les p-valeurs et l'interprétation des effets?

**Q** Interpréter l'influence des facteurs.

#### Avec tout qualitatif

In [ ]:
panel.glm=glm(THISYR~HUBINCq+AGEq+EDUCq+CHILD1+CHILD2+ASCEND,family=binomial,data=panel)
summary(panel.glm)
anova(panel.glm,test="Chi")

**Q** L'ajustement est-il meilleur?

**Q** L'interprétation de l'influence des facteurs reste-t-elle la même?

In [ ]:
#### Avec interactions

In [ ]:
# Supprimer les warnings pour simplifier la sortie
options(warn=-1)
panel.glm=glm(THISYR~(LHUBINC+AGE+EDUC+CHILD1+CHILD2+ASCEND)**2,family=binomial,data=panel)
panel.step=step(panel.glm,direction="backward", trace=0)
options(warn=0)
summary(panel.step)
anova(panel.step,test="Chi")

**Q** Quel problème (*warnings*) apparaît?

**Q** Que devient l'ajustement du modèle?

*Remarque*: Si l'ojectif est l'explication plutôt que la prévision, il serait utile de continuer à éliminer des termes du modèles (un par un) car beaucoup restent "approximativement" non significatifs et cela éviterait les *warnings*. Cette option utilisant le test de Ward ou celui du maximum de vraisemblance, plutôt que le critère AIC, est présente dans SAS, pas dans R. Il faudrait le faire "à la main".

**Q** Une interaction reste présente entre la variable `CHILD1` et celle `ASCEND`. Comment l'interpréter?

## 3 Modèle binomial : régression logistique pour prévision d'une coronopathie
### 3.1 Gestion des données
#### Données cliniques
Des données publiques disponibles sur le site [UCI repository](http://archive.ics.uci.edu/ml/) décrivent des facteurs de risque et résultats cliniques: 13 parmi 75 de l’étude originale de [Detrano et al. (1989)](http://www.ajconline.org/article/0002-9149%2889%2990524-9/abstract), liés à une maladie coronarienne (athérosclérose). Celle-ci est jugée présente lorsque tous les vaisseaux coronariens sont obstrués à plus de 50% par des athéromes. Les variables étudiées sont observées sur un échantillon de 270 patients admis dans une clinique de Cleveland (Ohio) à la suite de douleurs thoraciques pouvant être dues à une angine de poitrine. Elles sont décrites dans le tableau ci-dessous:

Num| Code | Libellé | Valeurs
-|--|--|--
1|`Age`   |  		
2|`Sexe`|   |sxF, sxM
3|`Douleur`|	Thoracique|	dlA (angine typique), dlb(atypique) dlc(différent) dlD(asymptom.)
4|`Tension`|	Systolique|	mmHg à l’admission et au repos
5|`Cholest`|	Taux|	mg/dl (préférable<200, limite entre 200 et 240, risqué au-delà)
6|`Sucre`|	Taux à jeun|	scN (<120mg/dl), scO (>120mg/dl)
7|`Cardio`|	ECG au repos|	cdA (Normal) cdB (ST/T anormal) cdC (hypertrophie ventr. gauche)
8|`FreqM`|	Fréquence|	cardiaque maximum lors du test d’effort
9|`AngInd`|	Angine induite|	par l’effort :  tmA (oui), tmB (non)
10|`PicInd`|	Dépression ST|	Induite par effort / repos
11|`PentInd`|	Segment ST| 	Induit à l’effort piA(ascendante), piB(plate), piC(descendante)  
12|`Nvais`|	Nombre de|	vaisseaux fl0, fl1, fl2, fl3 majeurs colorés par fluoroscopie
13|`Thal`|	Scintigraphie|	thN(normal) thF(défaut fixé) thR(défaut révers.) avec effort
14|`Coro`|	Coronaropathie|	CoA(Absence), CoP(Présence)

Certaines sont associées à des risques potentiels et d’autres sont résultats d’examens cliniques au repos ou à la suite d’un test d’effort. Les variables 1, 4, 5, 8, 10 sont quantitatives, les autres sont qualitatives dont certaines binaires : 2, 6, 9, 14. Le diagnostic (variable `Coro`) a été établi par une *angiographie* permettant de mesurer l’obstruction des artères coronariennes. 

Après la [phase exploratoire](https://github.com/wikistat/Exploration/blob/master/Diag-coro/Explo-R-DiagCoro.ipynb) l’objectif sur ces données est de construire un modèle de prévision de la variable `Coro` à partir de l’observation des autres, pas ou peu invasives, car l’angiographie est un **examen invasif** comportant des risques. C'est en fait plus qu'un examen puisqu'il permet par exemple de simultanément poser des *stents* pour faciliter le débit sanguin. 


*N.B.* Ce tutoriel est une version plus courte, limité à la régression logistique, de [celui](https://github.com/wikistat/Apprentissage/blob/master/Diag-coro/Apprent-R-DiagCoro.ipynb) qui propose une comparaison systématique de différentes méthodes et algorithmes d'apprentissage machine.

#### Préparation des données
Cette étape résume rapidement la suite des commandes qui découlent de la phase [phase exploratoire](https://github.com/wikistat/Exploration/blob/master/Diag-coro/Explo-R-DiagCoro.ipynb) des données qu'il est vivement conseillé d'avoir pratiquée pour se familiariser avec celles-ci.

Lecture des données et préparation pour construire le *dataFrame*. Les données sont lues et les codes des modalités (niveaux des facteurs) sont renommés de façon explicite.

In [ ]:
path="http://www.math.univ-toulouse.fr/~besse/Wikistat/data/"
#path=""
heart=read.table(paste(path,"heart.dat",sep=""))
# recodage des classes et nom des variables
heart=data.frame(
    Age=heart[,1],
    Sexe=factor(as.factor(heart[,2]),labels=c("sxF","sxM")),
    Douleur=factor(as.factor(heart[,3]),labels=c("dlA","dlB","dlC","dlD")),
    Tension=heart[,4],
    Cholest=heart[,5],
    Sucre=factor(as.factor(heart[,6]),labels=c("scN","scO")),
    Cardio=factor(as.factor(heart[,7]),labels=c("cdA","cdB","cdC")),
    FreqM=heart[,8],
    AngInd=factor(as.factor(heart[,9]),labels=c("tmA","tmB")),
    PicInd=heart[,10],
    PenteInd=factor(as.factor(heart[,11]),labels=c("piA","piB","piC")),
    Nvais=factor(as.factor(heart[,12]),labels=c("fl0","fl1","fl2","fl3")),
    Thal=factor(as.factor(heart[,13]),labels=c("thN","thF","thR")),
    Coro=factor(as.factor(heart[,14]),labels=c("CoA","CoP")))
summary(heart)

Découpage des variables quantitatives en trois classes pour permettre une analyse factorielle multiple des correspondances.

In [ ]:
AgeQ=cut(heart$Age,breaks=quantile(heart$Age,c(0,.33,.66,1)),labels=c("AgeA","AgeB","AgeC"),include.lowest = TRUE)
TensQ=cut(heart$Tension,breaks=quantile(heart$Tension,c(0,.33,.66,1)),labels=c("TensA","TensB","TensC"),include.lowest = TRUE)
CholQ=cut(heart$Cholest,breaks=quantile(heart$Cholest,c(0,.33,.66,1)),labels=c("CholA","CholB","CholC"),include.lowest = TRUE)
FreQ=cut(heart$FreqM,breaks=quantile(heart$FreqM,c(0,.33,.66,1)),labels=c("FreqA","FreqB","FreqC"),include.lowest = TRUE)
PicQ=cut(heart$PicInd,breaks=quantile(heart$PicInd,c(0,.33,.66,1)),labels=c("PicA","PicB","PIcC"),include.lowest = TRUE)

Suppression d'une modalité trop rare.

In [ ]:
heart[heart$Cardio=="cdB","Cardio"]="cdA"
heart$Cardio=factor(heart$Cardio,exclude=NULL)

Création finale du *dataFrame*

In [ ]:
heartT=data.frame(heart,AgeQ,TensQ,CholQ,FreQ,PicQ)
summary(heartT)

#### Analyse exploratoire des variables rendues qualitatives
L'étude exploratoire montre l'importance des variables qualitatives. en particulier, l'âge n'intervient pas comme risque de la même façon en fonction du genre des patients. Aussi, une nouvelle variable croisant sexe et âge est construite et une version simplifiée, avec 3 modalités au lieu de 6, est ajoutée à la base.

In [ ]:
# Sélection des seules variables qualitatives
heartQ=heartT[,-c(1,4,5,8,10)]
summary(heartQ)

In [ ]:
# Nouvelle variable pour intégrer l'interaction Sexe x Age
SexAge=heartQ$Sexe:heartQ$AgeQ
heartQ=data.frame(heartQ,"SexAge"=SexAge)

In [ ]:
# Simplifier la variable en regroupant les jeunes, hommes et femmes, dans la même classe
SxAg=as.factor(rep(c("sHFageA","sFageBC","sHageBC"),90))
SxAg[heartQ$AgeQ=="AgeA"]="sHFageA"
SxAg[heartQ$SexAge=="sxF:AgeB" | heartQ$SexAge=="sxF:AgeC"]="sFageBC"
SxAg[heartQ$SexAge=="sxM:AgeB" | heartQ$SexAge=="sxM:AgeC"]="sHageBC"

In [ ]:
# Complétion du data frame
heartQ2=data.frame(heartQ[,-15],SxAg)
summary(heartQ2)

#### Ajout de variables ou caractéristiques (*features*)
Cette étude teste la stratégie suivante: ajouter des nouvelles variables ou *features* sous la forme des composantes principales issues de l'AFCM du tableau disjoinctif complet. Si toutes les variables étaient quantitatives, cela reviendrait à ajouter les composantes principales d'une analyse en composantes principales mais en prenant toutes celles qualitatives plus les quantitatives découpées en classes, c'est une AFCM qui convient.

Cette approche a un autre avantage, elle permet de considérer des méthodes de classification supervisée qui n'autorisent que des variables quantitatives comme variables prédictives tout en conservant l'information apportée par les variables qualitatives. 

**Q** Quelle autre approche peut être mise en oeuvre pour contourner ce problème?

La pratique qui consiste à ajouter de nouvelles caractéristiques des données est habituelles en apprentissage machine, notamment dans les concours de type [kaggle](https://www.kaggle.com/) pour tenter d'améliorer la qualité de prévision.

#### Construction des composantes principales
Les nouvelles variables (composantes principales) sont construites à l'aide d'une AFCM disponible dans la librairie *FactoMineR*. Elles sont encore les composantes principales de l'ACP, avec la métrique dite du Chi2, du tableau disjonctif complet.

Une première exécution fournit la représentation graphique de toutes les modalités dans le premier plan factoriel.

In [ ]:
library(FactoMineR)
afc=MCA(heartQ2,quali.sup=c(1,9,10),graph=F)
options(repr.plot.width=5, repr.plot.height=5)
par(mar=c(2.1, 2.1, .1, .1))
plot.MCA(afc,invisible=c("ind"),cex=0.8,habillage="quali",palette=palette(rainbow(14)),title="")

L'interprétation de ce graphique était un des objectifs de la [phase exploratoire](https://github.com/wikistat/Exploration/blob/master/Diag-coro/Explo-R-DiagCoro.ipynb). Remarquer simplement la position des modalités liées à la présence ou non de la pathologie: `CoP, CoA`, et celles représentant les facteurs de risque, dont `sHageBC` (hommes plus âgés), alors que le cholestérol (`CholA, B, C`) augmente avec l'âge (tous genres confondus) sans être nécessairement lié avec l'aggravation du risque.

In [ ]:
# Regrouper toutes les variables dans un même dataFrame
heartT=data.frame(heart,heartQ2[,c(10:15)])
summary(heartT)

### 3.2  Construction des échantillons
Un objectif est ici de déterminer si l'ajout de nouvelles "*features*": les composantes principales de l'afcm, ont un intérêt ou non pour améliorer la prévision.

#### Séparation des échantillons apprentissage et test
**Q** Qu'est ce qui nécessite cette démarche?

Choisir ci-dessous une initialisation différente du générateur de nombre aléatoire (XX).

In [ ]:
XX=11

In [ ]:
# séparation apprentissage / test
set.seed(XX)
npop=nrow(heartT)

ntest=100
testi=sample(1:npop,ntest)
appri=setdiff(1:npop,testi)

datapp=heartT[appri,]
datest=heartT[testi,]

#### Ajout des **features**
Les étapes suivantes calculent les composantes principales de l'AFCM ou coordonnées des individus dans la base des 
vecteurs propres. 

**Q** Les individus, patients, de l'étude appartenant à l'échantillon test sont considérés comme *individus supplémentaires*. Pourquoi?

In [ ]:
library(FactoMineR)
summary(heartQ)  
afc=MCA(heartQ2[,-9],ind.sup=testi,ncp=9,graph=F)

In [ ]:
summary(afc$ind$coord)

In [ ]:
summary(afc$ind.sup$coord)

#### Construction des bases de données de l'échantillon d'apprentissage et de test

In [ ]:
# Ajout des nouvelles variables
datapp=data.frame(datapp,afc$ind$coord)
datest=data.frame(datest,afc$ind.sup$coord)

In [ ]:
summary(datapp)

### 3.3 Modélisation par régression logistique linéaire
Le problème à résoudre est la prévision de la présence d'une coronopathie. C'est donc un problème de discrimination binaire que beaucoup de méthodes peuvent aborder. L'objectif est explicatif pour mieux comprendre la maladie et identifier les facteurs de risque mais, dans l'intérêt du patient, il peut n'être que prédictif par l'utilisation d'un modèle ou algorithme "boîte noire" si celui-ci fournit de meilleures prévisions.

Les premiers modèles sont linéaires, sans interactions.

La sélection de variables et donc la complexité du modèle sont optimisées avant de calculer l'erreur de prévision sur l'échantillon test et ce pour différents modèles. Retenir le "meilleur".

**Q** Quels sont les principaux critères et algorithmes de sélection de variables utilisables pour cet objectif? 

**Q** Quels sont ceux mis en oeuvre ci-dessous?

#### Sur les seules variables initiales quantitatives et qualitatives

In [ ]:
res0.log=glm(Coro~1,data=datapp,family="binomial")
# Changer le paramètre "trace" pour avoir la liste des modèles testés
res0.log.step=step(res0.log,scope=list(lower=~1,upper=~(Age+Sexe+Douleur+Tension+Cholest+
                  AngInd+PicInd+PenteInd+Nvais+Thal+Sucre+Cardio+FreqM+SxAg),
                  direction="both"),trace=0)
# Résultat des tests
anova(res0.log.step, test="Chisq")

**Q** Commenter la liste des variables sélectionées, les p-valeurs des tests.

In [ ]:
print(res0.log.step)

**Q** Interpréter les facteurs de risque. Est-ce compatible avec la représentation de l'AFCM?

La qualité de prévision du modèle est estimée par la prévision de l'échantillon test et mise en évidence par le calcul de la *matrice de confusion*. 

In [ ]:
# Prévision de l'échantillon test
table(predict(res0.log.step,datest,type="response")>0.5,datest[,14]=="CoP") 

**Q** Que dire de la qualité de ce modèle? De la forme de la matrice? Du choix du seuil égal à 0.5?

#### Sur l'ensemble des variables

In [ ]:
res.log=glm(Coro~1,data=datapp,family="binomial")
res.log.step=step(res.log,scope=list(lower=~1,upper=~(Age+Sexe+Douleur+Tension+Cholest+
                AngInd+PicInd+PenteInd+Nvais+Thal+Sucre+Cardio+FreqM+SxAg+Dim.1+Dim.2+Dim.3+
                                    Dim.4+Dim.5+Dim.6+Dim.8+Dim.9)),direction="both",trace=0)
anova(res.log.step, test="Chisq")
table(predict(res.log.step,datest,type="response")>0.5,datest[,14]=="CoP") 

**Q** L'ajout des composantes principales semble-t-il pertinent?
#### Sur les seules composantes de l'AFCM

In [ ]:
res.log=glm(Coro~1,data=datapp,family="binomial")
res.log.step=step(res.log,scope=list(lower=~1,upper=~(Dim.1+Dim.2+Dim.3+Dim.4+
                            Dim.5+Dim.6+Dim.8+Dim.9)),direction="both",trace=0)
anova(res.log.step, test="Chisq")
table(predict(res.log.step,datest,type="response")>0.5,datest[,14]=="CoP") 

**Q** Que pensez de la capacité des composantes pricipales de l'AFCM à résumer l'information pertinente apportée par les variables? Justifier.

**Q** Que perd on dans cette démarche?

Identifier le meilleur modèle en terme de prévision.

### 3.4  Modèlisation par régression logistique quadratique
Les modèles précédents en prennent pas en compte de possibles interactions entre les variables à l'exception de celle `age x sexe` introduite à la main. Les commandes suivantes recherchent directement un modèle avec interactions au plus d'ordre 2.

Caractériser la stratégie mise en oeuvre.

In [ ]:
res.log=glm(Coro~1,data=datapp,family="binomial")
# Supprimer les warnings pour simplifier la sortie
options(warn=-1)
res.log.step=step(res.log,scope=list(lower=~1,upper=~(Age+Sexe+Douleur+Tension+Cholest+
                  AngInd+PicInd+PenteInd+Nvais+Thal+Sucre+Cardio+FreqM+SxAg)**2,
                  direction="both"),trace=0)
# Résultat des tests
anova(res.log.step, test="Chisq")
# remettre les warnings
options(warn=0)

In [ ]:
table(predict(res.log.step,datest,type="response")>0.5,datest[,14]=="CoP") 

**Q** Commentaire sur les interactions sélectionnées?

**Q** Le modèle quadratique est-il  meilleur?

### 3.5 Arbre de décision 
A titre illustratif, un autre modèle est testé et qui prend en compte, par construction, les iteractions. Il s'agit d'un [arbre binaire de discrimination](http://wikistat.fr/pdf/st-m-app-cart.pdf).

Ce modèle est estimé sur les données initiales

In [ ]:
library(rpart)
library(partykit)
res.tree=rpart(Coro~.,data=datapp[,1:20],cp=0)
par(mar=c(2.1, 2.1, .1, .1))
plot(as.party(res.tree),gp = gpar(fontsize = 11))

**Q** Que dire de la facilité d'interprétation, des facteurs de risque identifiés par ce modèle? 

In [ ]:
table(predict(res.tree,datest,type="class"),datest[,14])

**Q** Que dire de la qualité de la prévision? 

### 3.6 Comparaison des courbes ROC
Retenir le meilleur modèle de régression logistique et l'introduire dans le code ci-dessous. Attention ce résultat dépend de la séparation initiale de l'échantillon entre apprentissage et test; il ne correspond pas nécessairement au choix réalisé ci-dessous.

In [ ]:
library(ROCR)
roclogit=predict(res0.log.step,newdata=datest,type="response")
predlogit=prediction(roclogit,datest[,14])
perflogit=performance(predlogit,"tpr","fpr")

roctree=predict(res.tree,newdata=datest,type="prob")[,2]
predtree=prediction(roctree,datest[,14])
perftree=performance(predtree,"tpr","fpr")


options(repr.plot.width=4, repr.plot.height=3)
par(mar=c(2.1, 2.1, .1, .1))
palette("default")
plot(perflogit,col=1,lwd=2,print.cutoffs.at=c(0.1, 0.5))
plot(perftree,col=2,lwd=2,add=T,print.cutoffs.at=c(0.1, 0.5, 0.98))

legend("bottomright",legend=c("logit","rpart"),
       y.intersp=2,col=c(1,2),lty=1,lwd=2,text.width=0.15,ncol=2)

**Q** Que dire de façon générale sur le choix de la meilleure stratégie pour déterminer le paquet de variables à utiliser? Distinguer selon l'objectif de prévision ou d'interprétation. 

**Q** Que signifient les valeurs présentes sur les courbes ROC? 

**Q** Quelle conclusion tirer de ce graphique en fonction du taux de faux positif acceptable?

**Remarque** importante: l'échantillon test est de **petite taille** et donc l'estimation de l'erreur sujette à caution (faible précision) car affectée d'une variance importante. Une estimaiton plus précise de la distribution de cette erreur (validation croisée *Monte Carlo*) ainsi qu'une comparaison systématique avec les autres méthodes et algorithmes d'apprentissage (*k*-nn, neurones, *boosting, random forest*, SVM) sont proposées dans un autre [tutoriel](https://github.com/wikistat/Apprentissage/blob/master/Diag-coro/Apprent-R-DiagCoro.ipynb).